In [1]:
import csv
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import pickle

In [2]:
badwords = ['nigger', 'fuck', 'sex', 'negro', 'cunt', 'bitch', 'cum', 'gay', 'dyke', 'fag', 'faggot']
redditwords = ['Edit:', 'edit:']


def check_bad_word(badwordlist, sentence):
    sentencecopy = sentence.split(' ')
    for word in badwords:
        if word in sentencecopy:
            return False
    return True

Read in files

In [3]:
kaggle_train = pd.read_csv("kaggle_train.csv")
kaggle_dev = pd.read_csv("kaggle_dev.csv")
kaggle_test = pd.read_csv("kaggle_test.csv")
prev_test = pd.read_csv("test_data.csv")
prev_train = pd.read_csv("train_data.csv")

In [29]:
pos = pd.read_csv("all_jokes.csv")
neg = pd.read_csv("all_negatives.csv")

Save function (into tsv)

In [11]:
def write_csv(posts, name):
    name = name + ".tsv"
    posts.to_csv(name, index=False, sep='\t')

put together true kaggle examples and previous examples

In [27]:
kaggle_trues = (kaggle_train[kaggle_train['humor'] == True]).append((kaggle_dev[kaggle_dev['humor'] == True]),ignore_index=True)
kaggle_trues= kaggle_trues.rename(columns={"humor":"label"})
kaggle_trues['label'] = kaggle_trues['label'].astype(int)

pos = pos.append(kaggle_trues,ignore_index=True)


put together negative kaggle examples and previous examples

In [40]:
kaggle_negs = (kaggle_train[kaggle_train['humor'] == False]).append((kaggle_dev[kaggle_dev['humor'] == False]),ignore_index=True)
kaggle_negs= kaggle_negs.rename(columns={"humor":"label"})
kaggle_negs['label'] = kaggle_negs['label'].astype(int)

neg = neg.append(kaggle_trues,ignore_index=True)

The total number of examples:

In [44]:
print(len(pos),len(neg))

177880 149515


pull out the same number of pos/neg examples from the entire dataset to create test dataset

In [47]:
include_idx = np.random.choice(np.arange(len(pos)), size=1000)
mask = np.zeros(pos.shape,dtype=bool)
mask[include_idx] = True
p_test = pos[mask]  # array([0, 1, 2, 3])
p_train = pos[~mask]

In [78]:
include_idx = np.random.choice(np.arange(len(neg)), size=(len(pos)+len(neg))//10)
mask = np.zeros(neg.shape,dtype=bool)
mask[include_idx] = True
n_test = neg[mask]  # array([0, 1, 2, 3])
n_train = neg[~mask]

In [79]:
test = p_test.append(n_test)
total_train = p_train.append(n_train)

In [90]:
msk = np.random.rand(len(total_train)) < (1/9)
dev = total_train[msk]
train = total_train[~msk]

In [95]:
write_csv(test, "total_test")
write_csv(train,"total_train")
write_csv(dev,"total_dev")

Create stats for the test train dev:

In [91]:
print(len(test), len(train),len(dev))

60804 528047 65939


In [100]:
a,b,c=(len(test), len(train),len(dev))

In [101]:
tot = a+b+c
a = a/tot * 100
b = b/tot * 100
c = c/tot * 100

In [102]:
print(a,b,c)

9.286030635776356 80.64371783319842 10.070251531025214
